In [1]:
%matplotlib ipympl
from matplotlib import pylab as plt
import matplotlib
import molpx

import os
import sys
import glob
from itertools import cycle

import numpy as np
import pandas as pd

from os.path import join

import seaborn as sns

import mdtraj as md
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform

import pyemma

import random

from os.path import join

from collections import OrderedDict
from collections import defaultdict


print("Python version:\n{}\n".format(sys.version))
print("matplotlib version: {}".format(matplotlib.__version__))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("seaborn version: {}".format(sns.__version__))

Python version:
3.6.5 |Anaconda custom (64-bit)| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]

matplotlib version: 2.2.2
pandas version: 0.23.0
numpy version: 1.12.1
seaborn version: 0.8.1


In [2]:
topfile = './Trajectories/open10_dry.parm7'
from glob import glob
trajfile = 'toy_traj.nc'

In [3]:
ref1 = md.load('./Trajectories/closed10_dry.rst7', top='./Trajectories/closed10_dry.parm7')
ref2 = md.load('./Trajectories/open10_dry.rst7', top='./Trajectories/open10_dry.parm7')

In [4]:
feat = pyemma.coordinates.featurizer(topfile)
feat.add_minrmsd_to_ref(ref1)
feat.add_minrmsd_to_ref(ref2)
feat.describe()

['minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7f318e8e50f0>',
 'minrmsd to frame 0 of <mdtraj.Trajectory with 1 frames, 2808 atoms, 192 residues, and unitcells at 0x7f318e2100b8>']

In [5]:
feat.dimension()

2

In [6]:
inp = pyemma.coordinates.load(trajfile, stride=1, features=feat)

In [2]:
pyemma.plots.plot_free_energy(inp[:, 0], inp[:, 1])
plt.xlabel('RMSD to Open') 
plt.ylabel('RMSD to Closed');

NameError: name 'pyemma' is not defined

In [9]:
t = md.load(trajfile, top=topfile, stride=1)

In [10]:
mpx_wdg_box = molpx.visualize.FES(t,
                                 #MD_trajfiles,
                                 t.topology,
                                 inp,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 proj_labels='RMSD',
                                 #n_overlays=5,
                                )
mpx_wdg_box


MolPXHBox(children=(NGLWidget(count=110), FigureCanvasNbAgg()))

In [11]:
tica_obj = pyemma.coordinates.tica(data=inp, lag=20, var_cutoff=0.95)
print('TICA dimension ', tica_obj.dimension())

TICA dimension  2


In [12]:
tica_obj.cumvar

array([ 0.58919237,  1.        ])

In [13]:
Y = tica_obj.get_output()

In [14]:
mpx_wdg_box = molpx.visualize.FES(t,
                                 #MD_trajfiles,
                                 t.topology,
                                 Y,
                                 #Y,
                                 nbins=50,
                                 #proj_idxs=[1,2],
                                 proj_labels='TIC',
                                 #n_overlays=5,
                                )
mpx_wdg_box


MolPXHBox(children=(NGLWidget(count=133), FigureCanvasNbAgg()))

In [26]:
mpx_wdg_box = molpx.visualize.traj(t, 
                                   t.topology, 
                                   Y,
                                   #Y_filenames,                                          
                                   plot_FES = False,                                        
                                   dt = 0.01, tunits='ns',    
                                   #traj_selection = 1,
                                   #sharey_traj=False,
                                   #max_frames=100,
                                   proj_idxs=[0, 1],
                                   panel_height=2,    
                                   #proj_labels='TIC'
                                  )
mpx_wdg_box

MolPXHBox(children=(VBox(children=(Button(description='NGL widgets', layout=Layout(width='100%'), style=Button…

In [17]:
n_clusters = 250 

In [18]:
clustering = pyemma.coordinates.cluster_kmeans(Y,k=n_clusters)

27-06-18 14:29:45 pyemma.coordinates.clustering.kmeans.KmeansClustering[48] INFO     Algorithm did not reach convergence criterion of 1e-05 in 10 iterations. Consider increasing max_iter.


In [19]:
dtrajs = clustering.dtrajs

In [20]:
plt.figure()
pyemma.plots.plot_free_energy(np.vstack(Y)[:, 0], np.vstack(Y)[:, 1])
cc_x = clustering.clustercenters[:, 0]
cc_y = clustering.clustercenters[:, 1]
plt.plot(cc_x, cc_y, linewidth=0, marker='o', markersize=5, color='black')

FigureCanvasNbAgg()

In [22]:
its = pyemma.msm.timescales_msm(dtrajs, lags=200, nits=10)

In [28]:
plt.figure()
pyemma.plots.plot_implied_timescales(its, ylog=False, units='frames', linewidth=2)
#xlim(0, 40); ylim(0, 120);


FigureCanvasNbAgg()

In [51]:
its = pyemma.msm.timescales_msm(dtrajs, lags=100, nits=10, errors='bayes', n_jobs=-1)

In [54]:
plt.figure()
matplotlib.rcParams.update({'font.size': 14})
pyemma.plots.plot_implied_timescales(its, show_mean=False, ylog=False, units='frames', linewidth=2)
#xlim(0, 5); ylim(0.1,22);


FigureCanvasNbAgg()

In [55]:
msm_lag = 30
M = pyemma.msm.estimate_markov_model(dtrajs, msm_lag)
print('fraction of states used = ', M.active_state_fraction)
print('fraction of counts used = ', M.active_count_fraction)

fraction of states used =  0.996
fraction of counts used =  0.9999974767418681


In [56]:
plt.figure()
timescales_msm = M.timescales(20) # FIXME!
plt.plot(timescales_msm, linewidth=0, marker='o')

FigureCanvasNbAgg()

In [38]:
ck = M.cktest(4, mlags=11, err_est=True)

In [39]:
#plt.figure()
pyemma.plots.plot_cktest(ck, diag=True, figsize=(7,7), layout=(2,2), padding_top=0.1, 
                 y01=False, padding_between=0.3, dt=0.1, units='ns')


FigureCanvasNbAgg()

(<Figure size 700x700 with 4 Axes>,
 array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f3165e91518>,
         <matplotlib.axes._subplots.AxesSubplot object at 0x7f3165ed8908>]], dtype=object))

In [57]:
xall = np.vstack(Y)[:,0]
yall = np.vstack(Y)[:,1]
W = np.concatenate(M.trajectory_weights())

In [43]:
proj_ev_all = [np.hstack([M.eigenvectors_right()[:,i][dtraj] for dtraj in M.discrete_trajectories_full])
               for i in range(1, 1)]

In [44]:
fig, axes = subplots(1, 3, figsize=(16,4))
for i, ax in enumerate(axes):
    plot_sampled_function(xall, yall, proj_ev_all[i], ax=ax, cbar=False, cmap=cm.Blues)
    plot_labels(ax)

NameError: name 'subplots' is not defined

In [60]:
n_sets = 2
M.pcca(n_sets)
pcca_dist = M.metastable_distributions
membership = M.metastable_memberships  # get PCCA memberships
# memberships over trajectory
dist_all = [np.hstack([pcca_dist[i,:][dtraj] for dtraj in M.discrete_trajectories_full]) for i in range(n_sets)]
mem_all = [np.hstack([membership[:,i][dtraj] for dtraj in M.discrete_trajectories_full]) for i in range(n_sets)]

IndexError: index 249 is out of bounds for axis 0 with size 249